In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#출력 그림 크기 조절
sns.set(rc={'figure.figsize':(12,12)})
plt.style.use('ggplot')
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

import warnings
warnings.filterwarnings("ignore")

# for "5. Modeling with CatBoostRegressor"
from catboost import CatBoostRegressor
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, StratifiedKFold

In [2]:
# 데이터 로드
tr = pd.read_csv('data/train.csv')
te = pd.read_csv('data/test.csv')

**코드오류 데이터 제거**
- 'C2085', 'C1397', 'C2431', 'C1649','C1036', 'C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988'

In [3]:
#오류 단지 번호 리스트
err =list()
err = ['C2085', 'C1397', 'C2431', 'C1649','C1036', 'C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']

#오류 행 제거
for i in err:
    tr.drop(tr[tr["단지코드"] == i].index,inplace = True)

#### 서울특별시 데이터 학습 데이터셋에서 제거 (테스트 데이터셋에 존재하지 않음)

In [4]:
#서울특별시 데이터 학습 데이터셋에서 제거 (테스트 데이터셋에 존재하지 않음)
tr = tr.loc[tr.지역 != '서울특별시']

공급유형 == 임대상가 -> 임대보증금, 임대료 0으로 대치

In [5]:
#임대보증금, 임대료 임대상가 대치
tr.loc[tr.공급유형 == "임대상가","임대보증금"] = 0
tr.loc[tr.공급유형 == "임대상가","임대료"] = 0

te.loc[te.공급유형 == "임대상가","임대보증금"] = 0
te.loc[te.공급유형 == "임대상가","임대료"] = 0

In [6]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2819 entries, 8 to 2951
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          2819 non-null   object 
 1   총세대수                          2819 non-null   int64  
 2   임대건물구분                        2819 non-null   object 
 3   지역                            2819 non-null   object 
 4   공급유형                          2819 non-null   object 
 5   전용면적                          2819 non-null   float64
 6   전용면적별세대수                      2819 non-null   int64  
 7   공가수                           2819 non-null   float64
 8   자격유형                          2819 non-null   object 
 9   임대보증금                         2812 non-null   object 
 10  임대료                           2812 non-null   object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2612 non-null   float64
 12  도보 10분거리 내 버스정류장 수            2819 non-null   float64
 13  단지내

지하철, 버스정류장 이름 변경

In [7]:
#지하철, 버스정류장 이름 변경
tr.rename(columns={"도보 10분거리 내 지하철역 수(환승노선 수 반영)" : "지하철", "도보 10분거리 내 버스정류장 수" : "버스"}, inplace=True)
te.rename(columns={"도보 10분거리 내 지하철역 수(환승노선 수 반영)" : "지하철", "도보 10분거리 내 버스정류장 수" : "버스"}, inplace=True)

테스트데이터 자격유형 결측치 정의

In [8]:
te.loc[196,"자격유형"] = "A"
te.loc[258,"자격유형"] = "C"

In [9]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2819 entries, 8 to 2951
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   단지코드      2819 non-null   object 
 1   총세대수      2819 non-null   int64  
 2   임대건물구분    2819 non-null   object 
 3   지역        2819 non-null   object 
 4   공급유형      2819 non-null   object 
 5   전용면적      2819 non-null   float64
 6   전용면적별세대수  2819 non-null   int64  
 7   공가수       2819 non-null   float64
 8   자격유형      2819 non-null   object 
 9   임대보증금     2812 non-null   object 
 10  임대료       2812 non-null   object 
 11  지하철       2612 non-null   float64
 12  버스        2819 non-null   float64
 13  단지내주차면수   2819 non-null   float64
 14  등록차량수     2819 non-null   float64
dtypes: float64(6), int64(2), object(7)
memory usage: 352.4+ KB


임대보증금 결측치 train = 7, test = 3

In [10]:
for i in tr.loc[tr.임대료.isna(),"단지코드"].unique():
    display(tr.loc[tr.단지코드 == i])

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
2331,C1350,1401,아파트,대전광역시,공공분양,74.94,317,2.0,D,NaN,NaN,NaN,6.0,1636.0,2315.0
2332,C1350,1401,아파트,대전광역시,공공분양,74.94,137,2.0,D,NaN,NaN,NaN,6.0,1636.0,2315.0
2333,C1350,1401,아파트,대전광역시,공공분양,74.94,22,2.0,D,NaN,NaN,NaN,6.0,1636.0,2315.0
2334,C1350,1401,아파트,대전광역시,공공분양,84.94,164,2.0,D,NaN,NaN,NaN,6.0,1636.0,2315.0
2335,C1350,1401,아파트,대전광역시,공공분양,84.94,19,2.0,D,NaN,NaN,NaN,6.0,1636.0,2315.0
2336,C1350,1401,아파트,대전광역시,공공분양,84.96,26,2.0,D,NaN,NaN,NaN,6.0,1636.0,2315.0
2337,C1350,1401,아파트,대전광역시,공공분양,84.97,26,2.0,D,NaN,NaN,NaN,6.0,1636.0,2315.0
2338,C1350,1401,아파트,대전광역시,공공임대(10년),51.99,106,2.0,A,28013000,408600,NaN,6.0,1636.0,2315.0
2339,C1350,1401,아파트,대전광역시,공공임대(10년),59.91,13,2.0,A,37474000,519350,NaN,6.0,1636.0,2315.0
2340,C1350,1401,아파트,대전광역시,공공임대(10년),59.92,223,2.0,A,37385000,516130,NaN,6.0,1636.0,2315.0


In [11]:
for i in te.loc[te.임대료.isna(),"단지코드"].unique():
    display(te.loc[te.단지코드 == i])

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
73,C1006,1505,아파트,대전광역시,영구임대,26.37,358,27.0,C,5787000,79980,2.0,5.0,428.0
74,C1006,1505,아파트,대전광역시,영구임대,26.37,229,27.0,C,5787000,79980,2.0,5.0,428.0
75,C1006,1505,아파트,대전광역시,영구임대,26.37,574,27.0,C,5787000,79980,2.0,5.0,428.0
76,C1006,1505,아파트,대전광역시,영구임대,26.37,10,27.0,D,NaN,NaN,2.0,5.0,428.0
77,C1006,1505,아파트,대전광역시,영구임대,26.37,10,27.0,D,NaN,NaN,2.0,5.0,428.0
78,C1006,1505,아파트,대전광역시,영구임대,31.32,298,27.0,C,6873000,94990,2.0,5.0,428.0
79,C1006,1505,아파트,대전광역시,영구임대,52.74,6,27.0,D,NaN,NaN,2.0,5.0,428.0
80,C1006,1505,상가,대전광역시,임대상가,38.00,1,27.0,D,0,0,2.0,5.0,428.0
81,C1006,1505,상가,대전광역시,임대상가,38.00,1,27.0,D,0,0,2.0,5.0,428.0
82,C1006,1505,상가,대전광역시,임대상가,37.26,1,27.0,D,0,0,2.0,5.0,428.0


자격유형 D는 모두 임대보증금, 임대료가 0이다.

In [12]:
#따라서 자격유형 D의 임대보증금, 임대료는 모두 0으로 대치
tr.loc[tr.자격유형 == "D","임대보증금"] = 0
tr.loc[tr.자격유형 == "D","임대료"] = 0
te.loc[te.자격유형 == "D","임대보증금"] = 0
te.loc[te.자격유형 == "D","임대료"] = 0

임대료랑 임대보증금에 특이값이 존재했던거 같은데 => **특이값 추가적으로 확인**

In [13]:
for i in tr.loc[tr.임대료 == '-',"단지코드"].unique():
    display(tr.loc[tr.단지코드 == i])

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
2547,C1326,1934,아파트,부산광역시,국민임대,24.72,472,43.0,H,-,-,0.0,4.0,1670.0,1153.0
2548,C1326,1934,아파트,부산광역시,국민임대,24.79,104,43.0,H,-,-,0.0,4.0,1670.0,1153.0
2549,C1326,1934,아파트,부산광역시,국민임대,26.83,590,43.0,H,-,-,0.0,4.0,1670.0,1153.0
2550,C1326,1934,아파트,부산광역시,국민임대,37.70,464,43.0,H,-,-,0.0,4.0,1670.0,1153.0
2551,C1326,1934,아파트,부산광역시,국민임대,46.94,304,43.0,H,-,-,0.0,4.0,1670.0,1153.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
2680,C1786,480,아파트,강원도,행복주택,16.91,156,25.0,K,-,-,0.0,3.0,338.0,345.0
2681,C1786,480,아파트,강원도,행복주택,26.90,136,25.0,K,-,-,0.0,3.0,338.0,345.0
2682,C1786,480,아파트,강원도,행복주택,26.90,72,25.0,K,-,-,0.0,3.0,338.0,345.0
2683,C1786,480,아파트,강원도,행복주택,26.90,24,25.0,K,-,-,0.0,3.0,338.0,345.0
2684,C1786,480,아파트,강원도,행복주택,36.91,92,25.0,J,30560000,145160,0.0,3.0,338.0,345.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
2906,C2186,924,아파트,대구광역시,국민임대,29.17,238,0.0,H,-,-,0.0,8.0,664.0,744.0
2907,C2186,924,아파트,대구광역시,국민임대,29.34,38,0.0,H,-,-,0.0,8.0,664.0,744.0
2908,C2186,924,아파트,대구광역시,국민임대,37.43,220,0.0,H,-,-,0.0,8.0,664.0,744.0
2909,C2186,924,아파트,대구광역시,국민임대,46.51,188,0.0,H,25981000,248630,0.0,8.0,664.0,744.0
2910,C2186,924,아파트,대구광역시,영구임대,26.37,219,0.0,C,13126000,105870,0.0,8.0,664.0,744.0
2911,C2186,924,아파트,대구광역시,영구임대,26.54,13,0.0,C,13164000,106190,0.0,8.0,664.0,744.0


In [14]:
for i in te.loc[te.임대료 == '-',"단지코드"].unique():
    display(te.loc[te.단지코드 == i])

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
1005,C2152,120,아파트,강원도,영구임대,24.83,66,9.0,C,-,-,0.0,1.0,40.0
1006,C2152,120,아파트,강원도,영구임대,33.84,54,9.0,C,-,-,0.0,1.0,40.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
1007,C1267,675,아파트,경상남도,국민임대,24.87,28,38.0,H,6882000,104370,0.0,1.0,467.0
1008,C1267,675,아파트,경상남도,국민임대,24.99,90,38.0,H,6882000,104370,0.0,1.0,467.0
1009,C1267,675,아파트,경상남도,국민임대,24.99,12,38.0,H,6882000,104370,0.0,1.0,467.0
1010,C1267,675,아파트,경상남도,국민임대,46.72,68,38.0,H,24420000,219230,0.0,1.0,467.0
1011,C1267,675,아파트,경상남도,국민임대,46.72,112,38.0,H,24420000,219230,0.0,1.0,467.0
1012,C1267,675,아파트,경상남도,영구임대,22.86,96,38.0,C,10876000,89230,0.0,1.0,467.0
1013,C1267,675,아파트,경상남도,영구임대,22.86,14,38.0,C,10876000,89230,0.0,1.0,467.0
1014,C1267,675,아파트,경상남도,행복주택,16.94,50,38.0,L,-,-,0.0,1.0,467.0
1015,C1267,675,아파트,경상남도,행복주택,26.85,66,38.0,L,-,-,0.0,1.0,467.0
1016,C1267,675,아파트,경상남도,행복주택,26.85,8,38.0,L,-,-,0.0,1.0,467.0


In [15]:
tr = tr.replace('-',0)
te = te.replace('-',0)
tr.임대보증금 = tr.임대보증금.astype(float)
te.임대보증금 = te.임대보증금.astype(float)
tr.임대료 = tr.임대료.astype(float)
te.임대료 = te.임대료.astype(float)

In [16]:
tr.isnull().sum().to_frame()

,0
단지코드,0
총세대수,0
임대건물구분,0
지역,0
공급유형,0
전용면적,0
전용면적별세대수,0
공가수,0
자격유형,0
임대보증금,0


In [17]:
te.isnull().sum().to_frame()

,0
단지코드,0
총세대수,0
임대건물구분,0
지역,0
공급유형,0
전용면적,0
전용면적별세대수,0
공가수,0
자격유형,0
임대보증금,0


In [18]:
tr.지하철.value_counts()

0.0    2185
1.0     388
2.0      33
3.0       6
Name: 지하철, dtype: int64

In [19]:
tr.loc[tr.지하철.isna(),"단지코드"].unique()

array(['C1312', 'C1874', 'C1424', 'C2100', 'C2520', 'C1616', 'C1704',
       'C2258', 'C1068', 'C1983', 'C2216', 'C2644', 'C1005', 'C1004',
       'C1875', 'C2156', 'C1175', 'C2583', 'C1350'], dtype=object)

In [20]:
for i in tr.loc[tr.지하철.isna(),"단지코드"].unique():
    display(tr.loc[tr.단지코드 == i])

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
94,C1312,518,아파트,충청남도,국민임대,39.72,60,12.0,A,17460000.0,122210.0,NaN,3.0,527.0,359.0
95,C1312,518,아파트,충청남도,국민임대,39.98,89,12.0,A,17460000.0,122210.0,NaN,3.0,527.0,359.0
96,C1312,518,아파트,충청남도,국민임대,41.55,225,12.0,A,19954000.0,130940.0,NaN,3.0,527.0,359.0
97,C1312,518,아파트,충청남도,국민임대,46.90,143,12.0,A,28687000.0,149660.0,NaN,3.0,527.0,359.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
98,C1874,619,아파트,충청남도,영구임대,26.37,294,2.0,C,3141000.0,69900.0,NaN,2.0,97.0,62.0
99,C1874,619,아파트,충청남도,영구임대,26.37,149,2.0,C,3141000.0,69900.0,NaN,2.0,97.0,62.0
100,C1874,619,아파트,충청남도,영구임대,31.32,149,2.0,C,3731000.0,83020.0,NaN,2.0,97.0,62.0
101,C1874,619,상가,충청남도,임대상가,12.62,1,2.0,D,0.0,0.0,NaN,2.0,97.0,62.0
102,C1874,619,상가,충청남도,임대상가,17.40,1,2.0,D,0.0,0.0,NaN,2.0,97.0,62.0
103,C1874,619,상가,충청남도,임대상가,17.40,1,2.0,D,0.0,0.0,NaN,2.0,97.0,62.0
104,C1874,619,상가,충청남도,임대상가,22.89,1,2.0,D,0.0,0.0,NaN,2.0,97.0,62.0
105,C1874,619,상가,충청남도,임대상가,23.13,1,2.0,D,0.0,0.0,NaN,2.0,97.0,62.0
106,C1874,619,상가,충청남도,임대상가,23.13,1,2.0,D,0.0,0.0,NaN,2.0,97.0,62.0
107,C1874,619,상가,충청남도,임대상가,23.25,1,2.0,D,0.0,0.0,NaN,2.0,97.0,62.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
144,C1424,625,아파트,대전광역시,공공임대(50년),39.99,268,2.0,A,14247000.0,198980.0,NaN,7.0,517.0,179.0
145,C1424,625,아파트,대전광역시,공공임대(50년),49.95,119,2.0,A,20173000.0,225100.0,NaN,7.0,517.0,179.0
146,C1424,625,아파트,대전광역시,공공임대(50년),49.95,237,2.0,A,21203000.0,225100.0,NaN,7.0,517.0,179.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
147,C2100,880,아파트,대전광역시,국민임대,51.51,46,4.0,A,22307000.0,89210.0,NaN,5.0,736.0,671.0
148,C2100,880,아파트,대전광역시,국민임대,51.59,35,4.0,A,22307000.0,89210.0,NaN,5.0,736.0,671.0
149,C2100,880,아파트,대전광역시,국민임대,51.90,589,4.0,A,22307000.0,89210.0,NaN,5.0,736.0,671.0
150,C2100,880,아파트,대전광역시,국민임대,59.91,118,4.0,A,27885000.0,111520.0,NaN,5.0,736.0,671.0
151,C2100,880,아파트,대전광역시,국민임대,59.94,46,4.0,A,27885000.0,111520.0,NaN,5.0,736.0,671.0
152,C2100,880,아파트,대전광역시,국민임대,59.99,46,4.0,A,27885000.0,111520.0,NaN,5.0,736.0,671.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
171,C2520,970,아파트,대전광역시,공공임대(50년),39.69,960,2.0,A,13155000.0,177500.0,NaN,2.0,420.0,386.0


...

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
783,C1875,1003,아파트,충청남도,영구임대,26.37,596,5.0,C,3141000.0,69900.0,NaN,8.0,192.0,209.0
784,C1875,1003,아파트,충청남도,영구임대,26.37,239,5.0,C,3141000.0,69900.0,NaN,8.0,192.0,209.0
785,C1875,1003,아파트,충청남도,영구임대,31.32,149,5.0,C,3731000.0,83020.0,NaN,8.0,192.0,209.0
786,C1875,1003,상가,충청남도,임대상가,14.10,1,5.0,D,0.0,0.0,NaN,8.0,192.0,209.0
787,C1875,1003,상가,충청남도,임대상가,18.98,1,5.0,D,0.0,0.0,NaN,8.0,192.0,209.0
788,C1875,1003,상가,충청남도,임대상가,19.36,1,5.0,D,0.0,0.0,NaN,8.0,192.0,209.0
789,C1875,1003,상가,충청남도,임대상가,21.46,1,5.0,D,0.0,0.0,NaN,8.0,192.0,209.0
790,C1875,1003,상가,충청남도,임대상가,21.46,1,5.0,D,0.0,0.0,NaN,8.0,192.0,209.0
791,C1875,1003,상가,충청남도,임대상가,22.83,1,5.0,D,0.0,0.0,NaN,8.0,192.0,209.0
792,C1875,1003,상가,충청남도,임대상가,23.40,1,5.0,D,0.0,0.0,NaN,8.0,192.0,209.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
805,C2156,306,아파트,충청남도,국민임대,36.55,120,0.0,A,8670000.0,80500.0,NaN,1.0,246.0,204.0
806,C2156,306,아파트,충청남도,국민임대,39.57,120,0.0,A,11147000.0,90410.0,NaN,1.0,246.0,204.0
807,C2156,306,아파트,충청남도,국민임대,51.93,65,0.0,A,19819000.0,127580.0,NaN,1.0,246.0,204.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
839,C1175,468,아파트,충청남도,국민임대,46.90,198,15.0,A,21134000.0,128070.0,NaN,2.0,477.0,430.0
840,C1175,468,아파트,충청남도,국민임대,51.93,90,15.0,A,26258000.0,160100.0,NaN,2.0,477.0,430.0
841,C1175,468,아파트,충청남도,국민임대,59.88,180,15.0,A,34585000.0,204940.0,NaN,2.0,477.0,430.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
2285,C2583,1106,아파트,대전광역시,국민임대,36.85,24,21.0,A,11397000.0,193550.0,NaN,2.0,752.0,982.0
2286,C2583,1106,아파트,대전광역시,국민임대,36.85,70,21.0,A,11397000.0,193550.0,NaN,2.0,752.0,982.0
2287,C2583,1106,아파트,대전광역시,국민임대,36.93,604,21.0,A,11397000.0,193550.0,NaN,2.0,752.0,982.0
2288,C2583,1106,아파트,대전광역시,국민임대,36.93,24,21.0,A,11397000.0,193550.0,NaN,2.0,752.0,982.0
2289,C2583,1106,아파트,대전광역시,국민임대,46.78,72,21.0,A,26774000.0,254840.0,NaN,2.0,752.0,982.0
2290,C2583,1106,아파트,대전광역시,국민임대,46.78,72,21.0,A,26774000.0,254840.0,NaN,2.0,752.0,982.0
2291,C2583,1106,아파트,대전광역시,영구임대,21.97,60,21.0,I,7175000.0,69250.0,NaN,2.0,752.0,982.0
2292,C2583,1106,아파트,대전광역시,영구임대,26.90,60,21.0,I,8707000.0,84230.0,NaN,2.0,752.0,982.0
2293,C2583,1106,아파트,대전광역시,영구임대,26.90,36,21.0,I,8707000.0,84230.0,NaN,2.0,752.0,982.0
2294,C2583,1106,아파트,대전광역시,영구임대,26.90,72,21.0,I,8707000.0,84230.0,NaN,2.0,752.0,982.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
2331,C1350,1401,아파트,대전광역시,공공분양,74.94,317,2.0,D,0.0,0.0,NaN,6.0,1636.0,2315.0
2332,C1350,1401,아파트,대전광역시,공공분양,74.94,137,2.0,D,0.0,0.0,NaN,6.0,1636.0,2315.0
2333,C1350,1401,아파트,대전광역시,공공분양,74.94,22,2.0,D,0.0,0.0,NaN,6.0,1636.0,2315.0
2334,C1350,1401,아파트,대전광역시,공공분양,84.94,164,2.0,D,0.0,0.0,NaN,6.0,1636.0,2315.0
2335,C1350,1401,아파트,대전광역시,공공분양,84.94,19,2.0,D,0.0,0.0,NaN,6.0,1636.0,2315.0
2336,C1350,1401,아파트,대전광역시,공공분양,84.96,26,2.0,D,0.0,0.0,NaN,6.0,1636.0,2315.0
2337,C1350,1401,아파트,대전광역시,공공분양,84.97,26,2.0,D,0.0,0.0,NaN,6.0,1636.0,2315.0
2338,C1350,1401,아파트,대전광역시,공공임대(10년),51.99,106,2.0,A,28013000.0,408600.0,NaN,6.0,1636.0,2315.0
2339,C1350,1401,아파트,대전광역시,공공임대(10년),59.91,13,2.0,A,37474000.0,519350.0,NaN,6.0,1636.0,2315.0
2340,C1350,1401,아파트,대전광역시,공공임대(10년),59.92,223,2.0,A,37385000.0,516130.0,NaN,6.0,1636.0,2315.0


In [21]:
for i in te.loc[te.지하철.isna(),"단지코드"].unique():
    display(te.loc[te.단지코드 == i])

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
68,C1472,548,아파트,충청남도,국민임대,36.62,116,2.0,A,8738000.0,87380.0,NaN,1.0,442.0
69,C1472,548,아파트,충청남도,국민임대,39.51,56,2.0,A,9985000.0,99850.0,NaN,1.0,442.0
70,C1472,548,아파트,충청남도,국민임대,46.79,56,2.0,A,13731000.0,131070.0,NaN,1.0,442.0
71,C1472,548,아파트,충청남도,국민임대,46.90,86,2.0,A,13731000.0,131070.0,NaN,1.0,442.0
72,C1472,548,아파트,충청남도,국민임대,51.93,234,2.0,A,18726000.0,143550.0,NaN,1.0,442.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
100,C1083,427,아파트,대전광역시,국민임대,39.85,40,0.0,A,12483000.0,106090.0,NaN,6.0,384.0
101,C1083,427,아파트,대전광역시,국민임대,51.48,64,0.0,A,32459000.0,187260.0,NaN,6.0,384.0
102,C1083,427,아파트,대전광역시,국민임대,51.99,107,0.0,A,32459000.0,187260.0,NaN,6.0,384.0
103,C1083,427,아파트,대전광역시,국민임대,51.99,96,0.0,A,32459000.0,187260.0,NaN,6.0,384.0
104,C1083,427,아파트,대전광역시,국민임대,59.45,120,0.0,A,39950000.0,243430.0,NaN,6.0,384.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
168,C2177,1245,아파트,충청남도,영구임대,26.37,478,2.0,C,3141000.0,69900.0,NaN,1.0,187.0
169,C2177,1245,아파트,충청남도,영구임대,26.37,596,2.0,C,3141000.0,69900.0,NaN,1.0,187.0
170,C2177,1245,아파트,충청남도,영구임대,31.32,149,2.0,C,3731000.0,83020.0,NaN,1.0,187.0
171,C2177,1245,상가,충청남도,임대상가,15.00,1,2.0,D,0.0,0.0,NaN,1.0,187.0
172,C2177,1245,상가,충청남도,임대상가,15.00,1,2.0,D,0.0,0.0,NaN,1.0,187.0
173,C2177,1245,상가,충청남도,임대상가,16.57,1,2.0,D,0.0,0.0,NaN,1.0,187.0
174,C2177,1245,상가,충청남도,임대상가,18.38,1,2.0,D,0.0,0.0,NaN,1.0,187.0
175,C2177,1245,상가,충청남도,임대상가,18.38,1,2.0,D,0.0,0.0,NaN,1.0,187.0
176,C2177,1245,상가,충청남도,임대상가,20.90,1,2.0,D,0.0,0.0,NaN,1.0,187.0
177,C2177,1245,상가,충청남도,임대상가,20.90,1,2.0,D,0.0,0.0,NaN,1.0,187.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
305,C1318,312,아파트,충청남도,공공임대(50년),39.51,135,1.0,A,14440000.0,208840.0,NaN,11.0,226.0
306,C1318,312,아파트,충청남도,공공임대(50년),39.72,18,1.0,A,14515000.0,208050.0,NaN,11.0,226.0
307,C1318,312,아파트,충청남도,공공임대(50년),49.99,159,1.0,A,18269000.0,258880.0,NaN,11.0,226.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
891,C1327,98,아파트,대전광역시,행복주택,16.83,30,13.0,J,18800000.0,94000.0,NaN,17.0,98.0
892,C1327,98,아파트,대전광역시,행복주택,26.51,10,13.0,J,28400000.0,142000.0,NaN,17.0,98.0
893,C1327,98,아파트,대전광역시,행복주택,26.51,10,13.0,J,28400000.0,142000.0,NaN,17.0,98.0
894,C1327,98,아파트,대전광역시,행복주택,36.92,48,13.0,J,36800000.0,184000.0,NaN,17.0,98.0


In [22]:
tr.지하철 = tr.지하철.fillna(0)
te.지하철 = te.지하철.fillna(0)

버스 이상치 확인 = > **추가적으로 확인!**

In [23]:
for i in tr.loc[tr.버스 >= 12,"단지코드"].unique():
    display(tr.loc[tr.단지코드 == i])

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
172,C2319,606,아파트,대전광역시,국민임대,51.86,98,1.0,A,17389000.0,236220.0,1.0,12.0,548.0,266.0
173,C2319,606,아파트,대전광역시,국민임대,51.91,83,1.0,A,17405000.0,236470.0,1.0,12.0,548.0,266.0
174,C2319,606,아파트,대전광역시,국민임대,51.96,157,1.0,A,17423000.0,236710.0,1.0,12.0,548.0,266.0
175,C2319,606,아파트,대전광역시,국민임대,59.57,267,1.0,A,19975000.0,273790.0,1.0,12.0,548.0,266.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
439,C1497,802,아파트,부산광역시,공공임대(50년),39.41,357,7.0,A,13554000.0,171810.0,1.0,14.0,375.0,382.0
440,C1497,802,아파트,부산광역시,공공임대(50년),39.43,445,7.0,A,12953000.0,165340.0,1.0,14.0,375.0,382.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
640,C2644,779,아파트,충청남도,공공임대(50년),37.67,89,7.0,A,11879000.0,161250.0,0.0,12.0,190.0,177.0
641,C2644,779,아파트,충청남도,공공임대(50년),37.67,298,7.0,A,11879000.0,161250.0,0.0,12.0,190.0,177.0
642,C2644,779,아파트,충청남도,영구임대,26.37,239,7.0,C,8684000.0,127670.0,0.0,12.0,190.0,177.0
643,C2644,779,아파트,충청남도,영구임대,26.37,149,7.0,C,8684000.0,127670.0,0.0,12.0,190.0,177.0
644,C2644,779,상가,충청남도,임대상가,48.90,1,7.0,D,0.0,0.0,0.0,12.0,190.0,177.0
645,C2644,779,상가,충청남도,임대상가,49.72,1,7.0,D,0.0,0.0,0.0,12.0,190.0,177.0
646,C2644,779,상가,충청남도,임대상가,49.72,1,7.0,D,0.0,0.0,0.0,12.0,190.0,177.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
1129,C2094,410,아파트,경상남도,국민임대,36.22,90,16.0,A,13633000.0,101830.0,0.0,20.0,402.0,318.0
1130,C2094,410,아파트,경상남도,국민임대,36.22,36,16.0,A,13633000.0,101830.0,0.0,20.0,402.0,318.0
1131,C2094,410,아파트,경상남도,국민임대,39.15,64,16.0,A,14742000.0,107060.0,0.0,20.0,402.0,318.0
1132,C2094,410,아파트,경상남도,국민임대,39.15,33,16.0,A,14742000.0,107060.0,0.0,20.0,402.0,318.0
1133,C2094,410,아파트,경상남도,국민임대,46.47,97,16.0,A,17499000.0,116200.0,0.0,20.0,402.0,318.0
1134,C2094,410,아파트,경상남도,국민임대,46.48,60,16.0,A,17499000.0,116200.0,0.0,20.0,402.0,318.0
1135,C2094,410,아파트,경상남도,국민임대,46.48,30,16.0,A,17499000.0,116200.0,0.0,20.0,402.0,318.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
1136,C1880,635,아파트,경상남도,국민임대,51.86,422,15.0,A,22628000.0,131980.0,0.0,20.0,508.0,761.0
1137,C1880,635,아파트,경상남도,국민임대,51.92,31,15.0,A,22628000.0,131980.0,0.0,20.0,508.0,761.0
1138,C1880,635,아파트,경상남도,국민임대,51.96,66,15.0,A,22628000.0,131980.0,0.0,20.0,508.0,761.0
1139,C1880,635,아파트,경상남도,국민임대,59.99,116,15.0,A,27657000.0,148320.0,0.0,20.0,508.0,761.0


...

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
1783,C2378,818,아파트,경기도,국민임대,51.86,388,25.0,A,18092000.0,229990.0,0.0,12.0,640.0,901.0
1784,C2378,818,아파트,경기도,국민임대,51.88,270,25.0,A,18092000.0,229990.0,0.0,12.0,640.0,901.0
1785,C2378,818,아파트,경기도,국민임대,59.57,120,25.0,A,20677000.0,265190.0,0.0,12.0,640.0,901.0
1786,C2378,818,아파트,경기도,국민임대,59.57,40,25.0,A,20677000.0,265190.0,0.0,12.0,640.0,901.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
1787,C1268,1035,아파트,경기도,국민임대,36.72,276,21.0,A,15577000.0,138900.0,0.0,16.0,911.0,934.0
1788,C1268,1035,아파트,경기도,국민임대,39.48,108,21.0,A,15577000.0,160960.0,0.0,16.0,911.0,934.0
1789,C1268,1035,아파트,경기도,국민임대,46.96,171,21.0,A,18175000.0,194720.0,0.0,16.0,911.0,934.0
1790,C1268,1035,아파트,경기도,국민임대,46.96,282,21.0,A,18175000.0,194720.0,0.0,16.0,911.0,934.0
1791,C1268,1035,아파트,경기도,국민임대,51.86,108,21.0,A,23367000.0,237570.0,0.0,16.0,911.0,934.0
1792,C1268,1035,아파트,경기도,국민임대,59.57,90,21.0,A,25964000.0,280400.0,0.0,16.0,911.0,934.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
2010,C1328,992,아파트,경기도,국민임대,26.12,160,36.0,A,13221000.0,143510.0,0.0,15.0,871.0,770.0
2011,C1328,992,아파트,경기도,국민임대,36.55,144,36.0,A,18646000.0,207930.0,0.0,15.0,871.0,770.0
2012,C1328,992,아파트,경기도,국민임대,36.55,26,36.0,A,18646000.0,207930.0,0.0,15.0,871.0,770.0
2013,C1328,992,아파트,경기도,국민임대,36.64,207,36.0,A,18646000.0,207930.0,0.0,15.0,871.0,770.0
2014,C1328,992,아파트,경기도,국민임대,36.78,33,36.0,A,18646000.0,207930.0,0.0,15.0,871.0,770.0
2015,C1328,992,아파트,경기도,국민임대,46.49,69,36.0,E,123711000.0,64430.0,0.0,15.0,871.0,770.0
2016,C1328,992,아파트,경기도,국민임대,46.49,63,36.0,E,123711000.0,64430.0,0.0,15.0,871.0,770.0
2017,C1328,992,아파트,경기도,국민임대,46.61,92,36.0,E,123711000.0,64430.0,0.0,15.0,871.0,770.0
2018,C1328,992,아파트,경기도,국민임대,46.61,38,36.0,E,123711000.0,64430.0,0.0,15.0,871.0,770.0
2019,C1328,992,아파트,경기도,국민임대,51.75,128,36.0,E,144496000.0,75250.0,0.0,15.0,871.0,770.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
2382,C1136,984,아파트,경기도,국민임대,26.89,296,31.0,A,13118000.0,146230.0,0.0,19.0,865.0,763.0
2383,C1136,984,아파트,경기도,국민임대,36.72,296,31.0,A,17849000.0,221500.0,0.0,19.0,865.0,763.0
2384,C1136,984,아파트,경기도,국민임대,46.40,228,31.0,E,124970000.0,65080.0,0.0,19.0,865.0,763.0
2385,C1136,984,아파트,경기도,국민임대,51.76,164,31.0,E,139418000.0,72610.0,0.0,19.0,865.0,763.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
2459,C1636,632,아파트,경기도,공공임대(10년),74.85,128,7.0,A,66000000.0,590000.0,0.0,20.0,750.0,946.0
2460,C1636,632,아파트,경기도,공공임대(10년),84.97,74,7.0,A,79500000.0,630000.0,0.0,20.0,750.0,946.0
2461,C1636,632,아파트,경기도,공공임대(10년),84.97,48,7.0,A,79500000.0,630000.0,0.0,20.0,750.0,946.0
2462,C1636,632,아파트,경기도,공공임대(10년),84.98,382,7.0,A,79500000.0,630000.0,0.0,20.0,750.0,946.0


In [24]:
for i in te.loc[te.버스 >= 12,"단지코드"].unique():
    display(te.loc[te.단지코드 == i])

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
17,C1456,619,아파트,부산광역시,국민임대,33.40,82,18.0,A,19706000.0,156200.0,0.0,16.0,547.0
18,C1456,619,아파트,부산광역시,국민임대,39.57,115,18.0,A,26435000.0,192250.0,0.0,16.0,547.0
19,C1456,619,아파트,부산광역시,국민임대,39.57,35,18.0,A,26435000.0,192250.0,0.0,16.0,547.0
20,C1456,619,아파트,부산광역시,국민임대,39.94,15,18.0,A,26435000.0,192250.0,0.0,16.0,547.0
21,C1456,619,아파트,부산광역시,국민임대,46.90,24,18.0,A,36048000.0,234300.0,0.0,16.0,547.0
22,C1456,619,아파트,부산광역시,국민임대,46.90,86,18.0,A,36048000.0,234300.0,0.0,16.0,547.0
23,C1456,619,아파트,부산광역시,국민임대,46.90,86,18.0,A,36048000.0,234300.0,0.0,16.0,547.0
24,C1456,619,아파트,부산광역시,국민임대,51.93,132,18.0,A,43257000.0,264350.0,0.0,16.0,547.0
25,C1456,619,아파트,부산광역시,국민임대,59.96,44,18.0,A,55275000.0,312420.0,0.0,16.0,547.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
165,C2523,371,아파트,강원도,국민임대,36.65,98,2.0,A,14943000.0,126030.0,0.0,13.0,281.0
166,C2523,371,아파트,강원도,국민임대,46.90,208,2.0,A,21442000.0,178800.0,0.0,13.0,281.0
167,C2523,371,아파트,강원도,국민임대,51.93,65,2.0,A,25992000.0,215710.0,0.0,13.0,281.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
383,C1852,898,아파트,경기도,국민임대,39.59,219,6.0,H,23720000.0,197230.0,0.0,50.0,785.0
384,C1852,898,아파트,경기도,국민임대,39.72,56,6.0,A,23720000.0,197230.0,0.0,50.0,785.0
385,C1852,898,아파트,경기도,국민임대,39.72,44,6.0,A,23720000.0,197230.0,0.0,50.0,785.0
386,C1852,898,아파트,경기도,국민임대,46.90,46,6.0,A,37454000.0,259660.0,0.0,50.0,785.0
387,C1852,898,아파트,경기도,국민임대,46.90,22,6.0,A,37454000.0,259660.0,0.0,50.0,785.0
388,C1852,898,아파트,경기도,국민임대,46.90,225,6.0,H,37454000.0,259660.0,0.0,50.0,785.0
389,C1852,898,아파트,경기도,국민임대,51.93,136,6.0,H,45193000.0,308350.0,0.0,50.0,785.0
390,C1852,898,아파트,경기도,국민임대,51.93,44,6.0,H,45193000.0,308350.0,0.0,50.0,785.0
391,C1852,898,아파트,경기도,국민임대,59.61,106,6.0,H,61674000.0,370790.0,0.0,50.0,785.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
604,C2028,1267,아파트,경상남도,국민임대,51.93,119,26.0,A,24969000.0,149810.0,0.0,12.0,1280.0
605,C2028,1267,아파트,경상남도,국민임대,51.93,59,26.0,A,24969000.0,149810.0,0.0,12.0,1280.0
606,C2028,1267,아파트,경상남도,국민임대,59.37,232,26.0,A,31210000.0,187260.0,0.0,12.0,1280.0
607,C2028,1267,아파트,경상남도,국민임대,59.44,117,26.0,A,31210000.0,187260.0,0.0,12.0,1280.0
608,C2028,1267,아파트,경상남도,국민임대,59.88,740,26.0,A,31210000.0,187260.0,0.0,12.0,1280.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
646,C2653,879,아파트,경기도,국민임대,51.86,348,25.0,A,22876000.0,232570.0,0.0,16.0,802.0
647,C2653,879,아파트,경기도,국민임대,51.86,60,25.0,A,22876000.0,232570.0,0.0,16.0,802.0
648,C2653,879,아파트,경기도,국민임대,51.95,29,25.0,A,22876000.0,232570.0,0.0,16.0,802.0
649,C2653,879,아파트,경기도,국민임대,59.57,442,25.0,A,25419000.0,274510.0,0.0,16.0,802.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
675,C2076,1385,아파트,경상남도,국민임대,33.24,564,25.0,A,10651000.0,99080.0,0.0,14.0,1139.0
676,C2076,1385,아파트,경상남도,국민임대,39.63,163,25.0,A,13625000.0,130070.0,0.0,14.0,1139.0
677,C2076,1385,아파트,경상남도,국민임대,39.63,13,25.0,A,13625000.0,130070.0,0.0,14.0,1139.0
678,C2076,1385,아파트,경상남도,국민임대,39.98,300,25.0,A,13625000.0,130070.0,0.0,14.0,1139.0
679,C2076,1385,아파트,경상남도,국민임대,46.90,225,25.0,A,20439000.0,167230.0,0.0,14.0,1139.0
680,C2076,1385,아파트,경상남도,국민임대,55.83,120,25.0,A,30969000.0,185810.0,0.0,14.0,1139.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
707,C1835,1060,아파트,경기도,국민임대,26.12,210,37.0,A,13221000.0,143510.0,0.0,18.0,925.0
708,C1835,1060,아파트,경기도,국민임대,36.55,161,37.0,A,18646000.0,207930.0,0.0,18.0,925.0
709,C1835,1060,아파트,경기도,국민임대,36.55,53,37.0,A,18646000.0,207930.0,0.0,18.0,925.0
710,C1835,1060,아파트,경기도,국민임대,36.63,240,37.0,A,18646000.0,207930.0,0.0,18.0,925.0
711,C1835,1060,아파트,경기도,국민임대,46.61,232,37.0,E,123711000.0,64430.0,0.0,18.0,925.0
712,C1835,1060,아파트,경기도,국민임대,51.55,114,37.0,E,144496000.0,75250.0,0.0,18.0,925.0
713,C1835,1060,아파트,경기도,국민임대,51.75,50,37.0,E,144496000.0,75250.0,0.0,18.0,925.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
762,C2267,1191,아파트,경기도,국민임대,26.12,184,33.0,A,12285000.0,142800.0,0.0,15.0,1052.0
763,C2267,1191,아파트,경기도,국민임대,36.55,104,33.0,A,17325000.0,207900.0,0.0,15.0,1052.0
764,C2267,1191,아파트,경기도,국민임대,36.55,48,33.0,A,17325000.0,207900.0,0.0,15.0,1052.0
765,C2267,1191,아파트,경기도,국민임대,36.63,192,33.0,A,17325000.0,207900.0,0.0,15.0,1052.0
766,C2267,1191,아파트,경기도,국민임대,36.64,207,33.0,A,17325000.0,207900.0,0.0,15.0,1052.0
767,C2267,1191,아파트,경기도,국민임대,46.49,67,33.0,A,34755000.0,290850.0,0.0,15.0,1052.0
768,C2267,1191,아파트,경기도,국민임대,46.49,65,33.0,A,34755000.0,290850.0,0.0,15.0,1052.0
769,C2267,1191,아파트,경기도,국민임대,46.61,156,33.0,A,34755000.0,290850.0,0.0,15.0,1052.0
770,C2267,1191,아파트,경기도,국민임대,51.55,90,33.0,A,42420000.0,344400.0,0.0,15.0,1052.0
771,C2267,1191,아파트,경기도,국민임대,51.75,78,33.0,A,42420000.0,344400.0,0.0,15.0,1052.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
784,C2691,765,아파트,경기도,공공임대(10년),74.89,25,25.0,A,66000000.0,550000.0,0.0,19.0,910.0
785,C2691,765,아파트,경기도,공공임대(10년),74.95,150,25.0,A,66000000.0,550000.0,0.0,19.0,910.0
786,C2691,765,아파트,경기도,공공임대(10년),84.84,590,25.0,A,80000000.0,580000.0,0.0,19.0,910.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
891,C1327,98,아파트,대전광역시,행복주택,16.83,30,13.0,J,18800000.0,94000.0,0.0,17.0,98.0
892,C1327,98,아파트,대전광역시,행복주택,26.51,10,13.0,J,28400000.0,142000.0,0.0,17.0,98.0
893,C1327,98,아파트,대전광역시,행복주택,26.51,10,13.0,J,28400000.0,142000.0,0.0,17.0,98.0
894,C1327,98,아파트,대전광역시,행복주택,36.92,48,13.0,J,36800000.0,184000.0,0.0,17.0,98.0


In [25]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2819 entries, 8 to 2951
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   단지코드      2819 non-null   object 
 1   총세대수      2819 non-null   int64  
 2   임대건물구분    2819 non-null   object 
 3   지역        2819 non-null   object 
 4   공급유형      2819 non-null   object 
 5   전용면적      2819 non-null   float64
 6   전용면적별세대수  2819 non-null   int64  
 7   공가수       2819 non-null   float64
 8   자격유형      2819 non-null   object 
 9   임대보증금     2819 non-null   float64
 10  임대료       2819 non-null   float64
 11  지하철       2819 non-null   float64
 12  버스        2819 non-null   float64
 13  단지내주차면수   2819 non-null   float64
 14  등록차량수     2819 non-null   float64
dtypes: float64(8), int64(2), object(5)
memory usage: 352.4+ KB


In [26]:
te.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1022 entries, 0 to 1021
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   단지코드      1022 non-null   object 
 1   총세대수      1022 non-null   int64  
 2   임대건물구분    1022 non-null   object 
 3   지역        1022 non-null   object 
 4   공급유형      1022 non-null   object 
 5   전용면적      1022 non-null   float64
 6   전용면적별세대수  1022 non-null   int64  
 7   공가수       1022 non-null   float64
 8   자격유형      1022 non-null   object 
 9   임대보증금     1022 non-null   float64
 10  임대료       1022 non-null   float64
 11  지하철       1022 non-null   float64
 12  버스        1022 non-null   float64
 13  단지내주차면수   1022 non-null   float64
dtypes: float64(7), int64(2), object(5)
memory usage: 111.9+ KB


#### age 전처리

age = pd.read_csv("data/age_gender_info.csv")

sns.heatmap(age.corr(), annot=True)
plt.show()

# 10대 이하 청소년으로 묶기
age["u10"] = age.loc[:,"10대미만(여자)":"10대(남자)"].sum(axis=1)
list_temp = age.loc[:,"10대미만(여자)":"10대(남자)"].columns
age = age.drop(list_temp,axis=1)

# 30대 청소년으로 묶기
age["p30"] = age.loc[:,"30대(여자)":"30대(남자)"].sum(axis=1)
list_temp = age.loc[:,"30대(여자)":"30대(남자)"].columns
age = age.drop(list_temp,axis=1)

# 60대 이상 중년층으로 묶기
age["o60"] = age.loc[:,"60대(여자)":"100대(남자)"].sum(axis=1)
list_temp = age.loc[:,"60대(여자)":"100대(남자)"].columns
age = age.drop(list_temp,axis=1)

sns.heatmap(age.corr(), annot=True)
plt.show()

#### 면적 분리

In [27]:
tr['전용면적'] = tr['전용면적'] //10*10
tr['전용면적'] = np.where(tr['전용면적'] > 100, 100,tr['전용면적'])
tr['전용면적'] = np.where(tr['전용면적'] < 15, 15, tr['전용면적'])

In [28]:
te['전용면적'] = te['전용면적'] //10*10
te['전용면적'] = np.where(te['전용면적'] > 100, 100,te['전용면적'])
te['전용면적'] = np.where(te['전용면적'] < 15, 15, te['전용면적'])

In [29]:
codes = tr.단지코드.unique()
areas = np.sort(tr.전용면적.unique())

df_train_edited = pd.DataFrame()
columns = ['단지코드', '등록차량수', '총세대수', '지역', '공가수','지하철', '버스', '단지내주차면수']

for order, code in enumerate(codes):
    temp_by_code = tr.loc[tr.단지코드 == code].reset_index(drop=True)
    ## 원 계열 값 그냥 가져오기 
    df_train_edited.loc[order, columns] = temp_by_code.loc[0, columns]             
    df_train_edited.loc[order, "총임대가구수"] = temp_by_code.전용면적별세대수.sum()
   
#     for area in areas:
#         temp_by_code_areas = temp_by_code.loc[temp_by_code.전용면적==area].reset_index(drop=True)

#         if temp_by_code_areas.shape[0] !=0:
#             df_train_edited.loc[order, f'면적_{int(area)}'] = temp_by_code_areas.전용면적별세대수.sum() / temp_by_code_areas.총세대수[0]
#         else:
#             df_train_edited.loc[order, f'면적_{int(area)}'] = 0
            


In [30]:
codes = te.단지코드.unique()
areas = np.sort(te.전용면적.unique())

df_test_edited = pd.DataFrame()
columns = ['단지코드', '총세대수', '지역', '공가수','지하철', '버스', '단지내주차면수']

for order, code in enumerate(codes):
    temp_by_code = te.loc[te.단지코드 == code].reset_index(drop=True)
    ## 원 계열 값 그냥 가져오기 
    df_test_edited.loc[order, columns] = temp_by_code.loc[0, columns]             
    df_test_edited.loc[order, "총임대가구수"] = temp_by_code.전용면적별세대수.sum()
        
#     for area in areas:
#         temp_by_code_areas = temp_by_code.loc[temp_by_code.전용면적==area].reset_index(drop=True)

#         if temp_by_code_areas.shape[0] !=0:
#             df_test_edited.loc[order, f'면적_{int(area)}'] = temp_by_code_areas.전용면적별세대수.sum() / temp_by_code_areas.총세대수[0]
#         else:
#             df_test_edited.loc[order, f'면적_{int(area)}'] = 0

In [31]:
train = df_train_edited

In [32]:
test = df_test_edited

In [33]:
train = pd.merge(train,tr.groupby("단지코드").median().loc[:,"임대보증금":"임대료"],how = 'left', on = '단지코드')
test = pd.merge(test,te.groupby("단지코드").median().loc[:,"임대보증금":"임대료"],how = 'left', on = '단지코드')

In [34]:
def reshape_cat_features(data, cast_col, value_col):
    res = data.drop_duplicates(['단지코드', cast_col]).assign(counter=1).pivot(index='단지코드', columns=cast_col, values=value_col).fillna(0)
    res.columns.name = None
    res = res.rename(columns={col:cast_col+'_'+col for col in res.columns})
    return res

#### 공급유형 분리

In [35]:
tr.loc[tr.공급유형.isin(['공공임대(5년)', '공공분양', '공공임대(10년)', '공공임대(분납)']), '공급유형'] = '공공임대(5년/10년/분납/분양)'
te.loc[te.공급유형.isin(['공공임대(5년)', '공공분양', '공공임대(10년)', '공공임대(분납)']), '공급유형'] = '공공임대(5년/10년/분납/분양)'
tr.loc[tr.공급유형.isin(['장기전세', '국민임대']), '공급유형'] = '국민임대/장기전세'
te.loc[te.공급유형.isin(['장기전세', '국민임대']), '공급유형'] = '국민임대/장기전세'

In [36]:
reshape_cat_features(data=tr, cast_col='공급유형', value_col='counter')

,공급유형_공공임대(50년),공급유형_공공임대(5년/10년/분납/분양),공급유형_국민임대/장기전세,공급유형_영구임대,공급유형_임대상가,공급유형_행복주택
단지코드,,,,,,
C1000,0.0,0.0,1.0,0.0,0.0,0.0
C1004,0.0,0.0,0.0,1.0,1.0,0.0
C1005,0.0,0.0,1.0,0.0,0.0,0.0
C1013,0.0,0.0,1.0,0.0,0.0,0.0
C1014,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...
C2663,0.0,0.0,1.0,0.0,0.0,0.0
C2666,0.0,0.0,1.0,0.0,0.0,0.0
C2670,0.0,0.0,1.0,0.0,0.0,0.0


In [37]:
reshape_cat_features(data=te, cast_col='공급유형', value_col='counter')

,공급유형_공공임대(50년),공급유형_공공임대(5년/10년/분납/분양),공급유형_국민임대/장기전세,공급유형_영구임대,공급유형_임대상가,공급유형_행복주택
단지코드,,,,,,
C1003,0.0,0.0,0.0,0.0,0.0,1.0
C1006,0.0,0.0,0.0,1.0,1.0,0.0
C1016,0.0,0.0,1.0,0.0,0.0,0.0
C1019,0.0,0.0,1.0,0.0,0.0,0.0
C1030,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...
C2653,0.0,0.0,1.0,0.0,0.0,0.0
C2675,0.0,0.0,1.0,0.0,0.0,0.0
C2676,0.0,0.0,0.0,1.0,1.0,0.0


In [38]:
train = pd.merge(train,reshape_cat_features(data=tr, cast_col='공급유형', value_col='counter'),how = 'left', on = '단지코드')
test = pd.merge(test,reshape_cat_features(data=te, cast_col='공급유형', value_col='counter'),how = 'left', on = '단지코드')

#### 자격유형 분리

In [39]:
#자격유형 - 심종수 임의 추가****
tr.loc[tr.자격유형.isin(['J', 'L', 'K', 'N', 'M', 'O']), '자격유형'] = '행복주택_공급대상'
te.loc[te.자격유형.isin(['J', 'L', 'K', 'N', 'M', 'O']), '자격유형'] = '행복주택_공급대상'

tr.loc[tr.자격유형.isin(['H', 'B', 'E', 'G']), '자격유형'] = '국민임대/장기전세_공급대상'
te.loc[te.자격유형.isin(['H', 'B', 'E', 'G']), '자격유형'] = '국민임대/장기전세_공급대상'

tr.loc[tr.자격유형.isin(['C', 'I', 'F']), '자격유형'] = '영구임대_공급대상'
te.loc[te.자격유형.isin(['C', 'I', 'F']), '자격유형'] = '영구임대_공급대상'

In [40]:
train = pd.merge(train,reshape_cat_features(data=tr, cast_col='자격유형', value_col='counter'),how = 'left', on = '단지코드')
test = pd.merge(test,reshape_cat_features(data=te, cast_col='자격유형', value_col='counter'),how = 'left', on = '단지코드')

In [41]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 405 entries, 0 to 404
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   단지코드                     405 non-null    object 
 1   등록차량수                    405 non-null    float64
 2   총세대수                     405 non-null    float64
 3   지역                       405 non-null    object 
 4   공가수                      405 non-null    float64
 5   지하철                      405 non-null    float64
 6   버스                       405 non-null    float64
 7   단지내주차면수                  405 non-null    float64
 8   총임대가구수                   405 non-null    float64
 9   임대보증금                    405 non-null    float64
 10  임대료                      405 non-null    float64
 11  공급유형_공공임대(50년)           405 non-null    float64
 12  공급유형_공공임대(5년/10년/분납/분양)  405 non-null    float64
 13  공급유형_국민임대/장기전세           405 non-null    float64
 14  공급유형_영구임대                4

In [42]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 0 to 149
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   단지코드                     150 non-null    object 
 1   총세대수                     150 non-null    float64
 2   지역                       150 non-null    object 
 3   공가수                      150 non-null    float64
 4   지하철                      150 non-null    float64
 5   버스                       150 non-null    float64
 6   단지내주차면수                  150 non-null    float64
 7   총임대가구수                   150 non-null    float64
 8   임대보증금                    150 non-null    float64
 9   임대료                      150 non-null    float64
 10  공급유형_공공임대(50년)           150 non-null    float64
 11  공급유형_공공임대(5년/10년/분납/분양)  150 non-null    float64
 12  공급유형_국민임대/장기전세           150 non-null    float64
 13  공급유형_영구임대                150 non-null    float64
 14  공급유형_임대상가                1

#### age merge

In [44]:
# train = pd.merge(train,age,how = 'left', on = '지역')
# test = pd.merge(test,age,how = 'left', on = '지역')

In [45]:
train

,단지코드,등록차량수,총세대수,지역,공가수,지하철,버스,단지내주차면수,총임대가구수,임대보증금,...,공급유형_공공임대(5년/10년/분납/분양),공급유형_국민임대/장기전세,공급유형_영구임대,공급유형_임대상가,공급유형_행복주택,자격유형_A,자격유형_D,자격유형_국민임대/장기전세_공급대상,자격유형_영구임대_공급대상,자격유형_행복주택_공급대상
0,C2515,205.0,545.0,경상남도,17.0,0.0,3.0,624.0,545.0,18433000.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,C1407,1064.0,1216.0,대전광역시,13.0,1.0,1.0,1285.0,1216.0,27637000.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,C1945,730.0,755.0,경기도,6.0,1.0,3.0,734.0,755.0,23139000.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,C1470,553.0,696.0,전라북도,14.0,0.0,2.0,645.0,696.0,16821500.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,C1898,415.0,566.0,전라북도,9.0,0.0,6.0,517.0,566.0,16821500.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,C2586,57.0,90.0,제주특별자치도,7.0,0.0,3.0,66.0,90.0,26000000.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
401,C2035,246.0,492.0,강원도,24.0,0.0,1.0,521.0,492.0,15500000.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
402,C2020,19.0,40.0,부산광역시,7.0,1.0,2.0,25.0,40.0,28000000.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
403,C2437,16.0,90.0,충청북도,12.0,0.0,1.0,30.0,90.0,10346000.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [46]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2819 entries, 8 to 2951
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   단지코드      2819 non-null   object 
 1   총세대수      2819 non-null   int64  
 2   임대건물구분    2819 non-null   object 
 3   지역        2819 non-null   object 
 4   공급유형      2819 non-null   object 
 5   전용면적      2819 non-null   float64
 6   전용면적별세대수  2819 non-null   int64  
 7   공가수       2819 non-null   float64
 8   자격유형      2819 non-null   object 
 9   임대보증금     2819 non-null   float64
 10  임대료       2819 non-null   float64
 11  지하철       2819 non-null   float64
 12  버스        2819 non-null   float64
 13  단지내주차면수   2819 non-null   float64
 14  등록차량수     2819 non-null   float64
dtypes: float64(8), int64(2), object(5)
memory usage: 352.4+ KB


### 4.2 Randomforest


In [47]:
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor

In [48]:
train = train.set_index("단지코드")
test = test.set_index("단지코드")
train["지역"] = train["지역"].astype('category').cat.codes
test["지역"] = test["지역"].astype('category').cat.codes


In [49]:
train.head()

,등록차량수,총세대수,지역,공가수,지하철,버스,단지내주차면수,총임대가구수,임대보증금,임대료,...,공급유형_공공임대(5년/10년/분납/분양),공급유형_국민임대/장기전세,공급유형_영구임대,공급유형_임대상가,공급유형_행복주택,자격유형_A,자격유형_D,자격유형_국민임대/장기전세_공급대상,자격유형_영구임대_공급대상,자격유형_행복주택_공급대상
단지코드,,,,,,,,,,,,,,,,,,,,,
C2515,205.0,545.0,2,17.0,0.0,3.0,624.0,545.0,18433000.0,149760.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
C1407,1064.0,1216.0,6,13.0,1.0,1.0,1285.0,1216.0,27637000.0,164610.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
C1945,730.0,755.0,1,6.0,1.0,3.0,734.0,755.0,23139000.0,193230.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
C1470,553.0,696.0,11,14.0,0.0,2.0,645.0,696.0,16821500.0,117145.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
C1898,415.0,566.0,11,9.0,0.0,6.0,517.0,566.0,16821500.0,117145.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [50]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 405 entries, C2515 to C2532
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   등록차량수                    405 non-null    float64
 1   총세대수                     405 non-null    float64
 2   지역                       405 non-null    int8   
 3   공가수                      405 non-null    float64
 4   지하철                      405 non-null    float64
 5   버스                       405 non-null    float64
 6   단지내주차면수                  405 non-null    float64
 7   총임대가구수                   405 non-null    float64
 8   임대보증금                    405 non-null    float64
 9   임대료                      405 non-null    float64
 10  공급유형_공공임대(50년)           405 non-null    float64
 11  공급유형_공공임대(5년/10년/분납/분양)  405 non-null    float64
 12  공급유형_국민임대/장기전세           405 non-null    float64
 13  공급유형_영구임대                405 non-null    float64
 14  공급유형_임대상가                

# Modeling with Gradient Boosting Regressor

- Setting Data For Model

In [51]:
X = train.drop(columns = ['등록차량수'])
y = train['등록차량수']

- Hyper params Tuning

In [52]:
def objective(trial: Trial) -> float:
    params_cat = {
        "random_state": 42,
        "learning_rate": 0.05,
        "n_estimators": 10000,
        "verbose" : 1,
        "objective" : "MAE",
        "max_depth": trial.suggest_int("max_depth", 1, 16),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.8, 1.0),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
    }
    
    X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2)

    model = CatBoostRegressor(**params_cat)
    model.fit(
        X_tr,
        y_tr,
        eval_set=[(X_tr, y_tr), (X_val, y_val)],
        early_stopping_rounds=10,
        verbose=False,
    )

    cat_pred = model.predict(X_val)
    log_score = mean_absolute_error(y_val, cat_pred)
    
    return log_score

In [53]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="cat_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective, n_trials=10)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2021-07-26 00:21:07,756] A new study created in memory with name: cat_opt
[I 2021-07-26 00:21:08,123] Trial 0 finished with value: 124.34855471286888 and parameters: {'max_depth': 6, 'colsample_bylevel': 0.9901428612819833, 'subsample': 0.8123957592679836, 'min_child_samples': 62, 'max_bin': 246}. Best is trial 0 with value: 124.34855471286888.
[I 2021-07-26 00:21:08,250] Trial 1 finished with value: 129.55061729316498 and parameters: {'max_depth': 3, 'colsample_bylevel': 0.8116167224336399, 'subsample': 0.9063233020424546, 'min_child_samples': 62, 'max_bin': 413}. Best is trial 0 with value: 124.34855471286888.
[I 2021-07-26 00:21:08,335] Trial 2 finished with value: 101.13613385687395 and parameters: {'max_depth': 1, 'colsample_bylevel': 0.9939819704323989, 'subsample': 0.8827098485602951, 'min_child_samples': 25, 'max_bin': 254}. Best is trial 2 with value: 101.13613385687395.
[I 2021-07-26 00:21:08,448] Trial 3 finished with value: 127.04905434403035 and parameters: {'max_depth'

Best Score: 101.13613385687395
Best trial: {'max_depth': 1, 'colsample_bylevel': 0.9939819704323989, 'subsample': 0.8827098485602951, 'min_child_samples': 25, 'max_bin': 254}


In [54]:
cat_p = study.best_trial.params
cat = CatBoostRegressor(**cat_p)

- StratifiedK-Fold for Regression

In [55]:
y_cat = pd.cut(y, 10, labels=range(10))
skf = StratifiedKFold(5)

preds = []
for tr_id, val_id in skf.split(X, y_cat) : 
    X_tr = X.iloc[tr_id]
    y_tr = y.iloc[tr_id]
    
    cat.fit(X_tr, y_tr, verbose = 0)
    
    pred = cat.predict(test)
    preds.append(pred)
cat_pred = np.mean(preds, axis = 0)

In [56]:
sample = pd.read_csv('data/sample_submission.csv')
sample['num'] = cat_pred
sample.to_csv('sub/cat0726_2.csv', index=False)

In [57]:
sample

,code,num
0,C1072,719.785707
1,C1128,1169.670280
2,C1456,598.265268
3,C1840,516.463084
4,C1332,1043.473395
...,...,...
145,C2456,218.520227
146,C1266,438.798462
147,C2152,92.173316
148,C1267,319.596173
